In [ ]:
!pip install scikit-learn
!pip install numpy
!pip install open_cv
!pip install ultralytics
!pip install json
!pip install pandas
!pip install roboflow

ERROR: Could not find a version that satisfies the requirement open_cv (from versions: none)
ERROR: No matching distribution found for open_cv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.0/800.0 kB 12.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (1

In [ ]:
import cv2
import numpy as np
import os
import glob
import json
import numpy as np
import cv2
from ultralytics import YOLO  # YOLOv8 from the ultralytics package
from sklearn.metrics import precision_score, recall_score
import pprint
import os

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5M9F7yWepauM9EQcrnpW")
project = rf.workspace("sara-ramezani").project("car-xi0dq")
version = project.version(2)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Car-2 in coco:: 100%|██████████| 971/971 [00:00<00:00, 6149.16it/s]


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="2dbvWhTriZibV4l8adoE")
project = rf.workspace("vehicle-ug2tj").project("vehicle-yioma")
version = project.version(2)
dataset = version.download("coco")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to vehicle-2 in coco:: 100%|██████████| 92/92 [00:00<00:00, 3951.95it/s]


#preprocessing

1. Night Mode:



In [ ]:
def enhance_low_light_image(image_path):
    # Read the image from the given path
    image = cv2.imread(image_path)

    # Convert the image to the LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

    # Split the LAB image to different channels
    l, a, b = cv2.split(lab_image)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the L-channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Merge the CLAHE enhanced L-channel with the a and b channels
    merged_lab = cv2.merge((cl, a, b))

    # Convert the LAB image back to BGR color space
    enhanced_image = cv2.cvtColor(merged_lab, cv2.COLOR_Lab2BGR)

    # Apply Gamma Correction to reduce the darkness
    gamma = 1.2  # Change the value to adjust brightness
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    gamma_corrected_image = cv2.LUT(enhanced_image, table)

    # Apply a bilateral filter to reduce noise while keeping edges sharp
    final_image = cv2.bilateralFilter(gamma_corrected_image, d=9, sigmaColor=75, sigmaSpace=75)

    # final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2gray)

    # Save the final image to the specified output path
    cv2.imwrite('v1.jpg' , final_image)

    # Read the image in grayscale
    final_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Perform histogram equalization
    # equalized_image = cv2.equalizeHist(image)

    # Apply Laplacian filter for whole picture sharpening
    laplacian = cv2.Laplacian(final_image, cv2.CV_64F)
    # Convert Laplacian to the same type as processed_image
    laplacian = np.uint8(np.absolute(laplacian))
    sharpened_image = cv2.addWeighted(final_image , 1.5, laplacian, -0.5, 0)
    final_image = sharpened_image

    return final_image

2. **Day mode:**

In [ ]:
# Define the image processing function
def process_and_enhance_image(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check the maximum and minimum pixel values
    min_val, max_val = image.min(), image.max()

    # Decide whether to apply histogram equalization or stretching
    # if max_val - min_val < 255:
    # # Perform histogram stretching
    #     stretched_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #     processed_image = stretched_image
    # else:
    # # Perform histogram equalization
    #     equalized_image = cv2.equalizeHist(image)
    #     processed_image = equalized_image

    stretched_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    processed_image = stretched_image

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(50 , 50))
    processed_image = clahe.apply(image)

    # Apply Laplacian filter for whole picture sharpening
    laplacian = cv2.Laplacian(processed_image, cv2.CV_64F)
    # Convert Laplacian to the same type as processed_image
    laplacian = np.uint8(np.absolute(laplacian))
    sharpened_image = cv2.addWeighted(processed_image, 1.5, laplacian, -0.5, 0)
    final_image = sharpened_image
    # Apply Gaussian Blur for noise reduction
    # final_image = cv2.GaussianBlur(sharpened_image, (3, 3), 0)


    return final_image

In [ ]:
def process_images(input_folder, output_folder , night_mode= False):
    # Process and enhance the image
    if night_mode:
      final_image = enhance_low_light_image(input_folder)
    else:
      final_image = process_and_enhance_image(input_folder)

    # Save the final image
    base_name = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f'processed_{base_name}')
    cv2.imwrite(output_path, final_image)

    # Print the status
    print(f"Image {base_name} has been processed and saved as {output_path}")

    return output_path

In [ ]:
input_directory = '/content/Car-1/test'  # input directory path
output_directory = '/content/processed-night'  # output directory path
process_images(input_directory, output_directory , True)

Image Carsdrivingatnight-Trim_mp4-0313_jpg.rf.acc81b9f0dc55e7e7dd59571ad6fbcf7.jpg has been processed and saved as /content/processed-night/processed_Carsdrivingatnight-Trim_mp4-0313_jpg.rf.acc81b9f0dc55e7e7dd59571ad6fbcf7.jpg


#MODEL:
run model on annotated dataset and caculate RECALL and Percision and F1-score



In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5M9F7yWepauM9EQcrnpW")
project = rf.workspace("sara-ramezani").project("car-xi0dq")
version = project.version(1)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Car-1 in coco:: 100%|██████████| 971/971 [00:00<00:00, 5080.87it/s]


In [ ]:
def calculate_iou(box1, box2):
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])

    inter_area = max(0, x2_min - x1_max) * max(0, y2_min - y1_max)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

In [ ]:
# Function to match predictions with ground truth
def match_predictions(predictions, ground_truths, iou_threshold=0.5):
    TP, FP, FN = 0, 0, 0
    for gt in ground_truths:
        match_found = False
        for pred in predictions:
            iou = calculate_iou(gt, pred)
            if iou >= iou_threshold:
                TP += 1
                match_found = True
                break
        if not match_found:
            FN += 1
    FP = len(predictions) - TP
    return TP, FP, FN


In [ ]:
# Load COCO annotations

with open('/content/Car-1/test/_annotations.coco.json') as f:
    coco_data = json.load(f)

In [ ]:
# Create a dictionary to map image_id to its annotations
image_annotations = {}
for annotation in coco_data['annotations']:
    image_id = annotation['image_id']
    bbox = annotation['bbox']  # COCO format is [x, y, width, height]
    bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]  # Convert to [x1, y1, x2, y2]
    category_id = annotation['category_id']  # COCO category ID
    if image_id in image_annotations:
        image_annotations[image_id].append((bbox, category_id))
    else:
        image_annotations[image_id] = [(bbox, category_id)]
print(image_annotations)

{0: [([971, 364, 1044, 408], 1), ([285, 550, 397, 648], 1), ([1173, 460, 1279, 531], 1), ([707, 544, 841, 653], 1), ([862, 405, 955, 455], 1), ([684, 350, 738, 385], 1), ([863, 360, 948, 412], 1), ([497, 509, 593, 599], 1), ([546, 321, 591, 344], 1), ([379, 328, 435, 363], 1), ([461, 306, 491, 325], 1), ([578, 294, 633, 334], 1), ([426, 279, 460, 305], 1), ([691, 335, 752, 360], 1), ([263, 268, 291.33, 296.33], 1), ([300, 272, 330, 308.67], 1), ([328, 290, 368, 310], 1)], 1: [([313, 472, 412, 577], 1), ([649, 492, 747, 579], 1), ([888, 380, 970, 423], 1), ([432, 394, 494, 455], 1), ([968, 448, 1073, 513], 1), ([598, 443, 683, 507], 1), ([780, 454, 880, 536], 1), ([1034, 378, 1128, 435], 1), ([662, 320, 720, 355], 1), ([520, 372, 587, 426], 1), ([318, 334, 366, 367], 1), ([1171, 531, 1279, 623], 1), ([791, 380, 874, 433], 1), ([393, 366, 446, 407], 1), ([592, 301, 651, 342], 1), ([384, 321, 435, 363], 1), ([649, 397, 690, 446], 1), ([453, 319, 511, 356], 1), ([767, 338, 823, 376], 1), (

In [ ]:
# Load your YOLOv8 model
model = YOLO("yolov8x.pt")

100%|██████████| 131M/131M [00:00<00:00, 460MB/s]


In [ ]:
# List of images to evaluate
image_paths = {img['id']: img['file_name'] for img in coco_data['images']}
pprint.pp(image_paths)

{0: 'Carsdrivingatnight-Trim_mp4-0048_jpg.rf.cbc96967276970000cd5c707113a6ca0.jpg',
 1: 'Carsdrivingatnight-Trim_mp4-0904_jpg.rf.c87b1c3e01cbb6dcda35341c12b44b7b.jpg',
 2: 'Carsdrivingatnight-Trim_mp4-0514_jpg.rf.ce5875b5395d4d93e4e537fce22b1192.jpg',
 3: 'Carsdrivingatnight-Trim_mp4-0593_jpg.rf.c8ce15b28e715288bce3c5584307cfed.jpg',
 4: 'Carsdrivingatnight-Trim_mp4-0175_jpg.rf.c9420f5b027fa78f7ce4806345509663.jpg',
 5: 'Carsdrivingatnight-Trim_mp4-0234_jpg.rf.cb6aa7e3c23dc09b327b1b6a5ec8f1ef.jpg',
 6: 'Carsdrivingatnight-Trim_mp4-0206_jpg.rf.d0083af82776da05d92e6a5ac7f9de6e.jpg',
 7: 'Carsdrivingatnight-Trim_mp4-0351_jpg.rf.ce2bb549a79ea01c4d850a5bb432bc09.jpg',
 8: 'Carsdrivingatnight-Trim_mp4-0905_jpg.rf.c9d5c003aeb25c1a00e07977da1740fd.jpg',
 9: 'Carsdrivingatnight-Trim_mp4-0807_jpg.rf.ceb0b13a2ea146240b58c22624f90107.jpg',
 10: 'Carsdrivingatnight-Trim_mp4-0750_jpg.rf.cdbbccfbbfa8ee137280126150c17c76.jpg',
 11: 'Carsdrivingatnight-Trim_mp4-0274_jpg.rf.cfc25a061f73d0691208deced2632

In [ ]:
# Initialize dictionaries to store TP, FP, FN for each class
class_metrics = {'car': {'TP': 0, 'FP': 0, 'FN': 0},
                 'bus': {'TP': 0, 'FP': 0, 'FN': 0},
                 'truck': {'TP': 0, 'FP': 0, 'FN': 0}}

In [ ]:
# Directory to save predicted images
output_dir = '/content/processed-night'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Iterate through each image
#gets preprocessed image as input and gives to model in order to predict
for image_id, image_path in image_paths.items():
    img = cv2.imread(f"/content/Car-1/test/{image_path}")

    #preprocessing percedure
    temp =process_images(f"/content/Car-1/test/{image_path}", '/content/processed-night' , True )
    # print("paaaaaaaaaaaaaath: " , temp)
    img = cv2.imread(temp)

    results = model.predict(img)

    # Initialize lists to store predictions and ground truths for each class
    predictions_car = []
    predictions_bus = []
    predictions_truck = []
    ground_truths_car = []
    ground_truths_bus = []
    ground_truths_truck = []

    # Collect predictions by class
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            conf = box.conf[0]
            cls = box.cls[0]
            bbox = [x1, y1, x2, y2]
            if int(cls) == 2:  # Class 0 is 'car'
                predictions_car.append(bbox)
                color = (255, 0, 0)  # Blue color for car
            elif int(cls) == 5:  # Class 1 is 'bus'
                predictions_bus.append(bbox)
                color = (0, 255, 0)  # Green color for bus
            elif int(cls) == 7:  # Class 2 is 'truck'
                predictions_truck.append(bbox)
                color = (0, 0, 255)  # Red color for truck

            # Draw the bounding box on the image
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            label = f'{cls}: {conf:.2f}'
            cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Get ground truths from annotations
    annotations = image_annotations.get(image_id, [])
    for bbox, category_id in annotations:
        if category_id == 1:  # Category ID 1 corresponds to 'car' in COCO annotations
            ground_truths_car.append(bbox)
        elif category_id == 5:  # Category ID  corresponds to 'bus' in COCO annotations
            ground_truths_bus.append(bbox)
        elif category_id ==  3:  # Category ID 8 corresponds to 'truck' in COCO annotations
            ground_truths_truck.append(bbox)

    # Calculate TP, FP, FN for each class
    TP_car, FP_car, FN_car = match_predictions(predictions_car, ground_truths_car)
    TP_bus, FP_bus, FN_bus = match_predictions(predictions_bus, ground_truths_bus)
    TP_truck, FP_truck, FN_truck = match_predictions(predictions_truck, ground_truths_truck)

    # Update class metrics
    class_metrics['car']['TP'] += TP_car
    class_metrics['car']['FP'] += FP_car
    class_metrics['car']['FN'] += FN_car
    class_metrics['bus']['TP'] += TP_bus
    class_metrics['bus']['FP'] += FP_bus
    class_metrics['bus']['FN'] += FN_bus
    class_metrics['truck']['TP'] += TP_truck
    class_metrics['truck']['FP'] += FP_truck
    class_metrics['truck']['FN'] += FN_truck

    # Save the predicted image
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, img)

Image Carsdrivingatnight-Trim_mp4-0048_jpg.rf.cbc96967276970000cd5c707113a6ca0.jpg has been processed and saved as /content/processed-night/processed_Carsdrivingatnight-Trim_mp4-0048_jpg.rf.cbc96967276970000cd5c707113a6ca0.jpg

0: 384x640 11 cars, 62.2ms
Speed: 2.0ms preprocess, 62.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Image Carsdrivingatnight-Trim_mp4-0904_jpg.rf.c87b1c3e01cbb6dcda35341c12b44b7b.jpg has been processed and saved as /content/processed-night/processed_Carsdrivingatnight-Trim_mp4-0904_jpg.rf.c87b1c3e01cbb6dcda35341c12b44b7b.jpg

0: 384x640 18 cars, 62.0ms
Speed: 3.2ms preprocess, 62.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Image Carsdrivingatnight-Trim_mp4-0514_jpg.rf.ce5875b5395d4d93e4e537fce22b1192.jpg has been processed and saved as /content/processed-night/processed_Carsdrivingatnight-Trim_mp4-0514_jpg.rf.ce5875b5395d4d93e4e537fce22b1192.jpg

0: 384x640 10 cars, 1 truck, 62.0ms
Speed: 2.6ms preprocess, 62.0ms 

In [ ]:
# Calculate precision, recall, F1-score, and accuracy for each class
for class_name, metrics in class_metrics.items():
    TP = metrics['TP']
    FP = metrics['FP']
    FN = metrics['FN']

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

    print(f"Class: {class_name}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1_score}")
    print(f"Accuracy: {accuracy}")
    print("--------------------")

Class: car
Precision: 0.9294441050702504
Recall: 0.7509408353383923
F1-Score: 0.8307114826821361
Accuracy: 0.7104418373898325
--------------------
Class: bus
Precision: 0.0
Recall: 0
F1-Score: 0
Accuracy: 0.0
--------------------
Class: truck
Precision: 0.875
Recall: 0.7021276595744681
F1-Score: 0.7790893760539629
Accuracy: 0.638121546961326
--------------------


In [ ]:
# Overall precision, recall, F1-score, and accuracy across all classes
overall_TP = sum([metrics['TP'] for metrics in class_metrics.values()])
overall_FP = sum([metrics['FP'] for metrics in class_metrics.values()])
overall_FN = sum([metrics['FN'] for metrics in class_metrics.values()])

overall_precision = overall_TP / (overall_TP + overall_FP) if (overall_TP + overall_FP) > 0 else 0
overall_recall = overall_TP / (overall_TP + overall_FN) if (overall_TP + overall_FN) > 0 else 0
overall_f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
overall_accuracy = overall_TP / (overall_TP + overall_FP + overall_FN) if (overall_TP + overall_FP + overall_FN) > 0 else 0

print("Overall Metrics:")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1-Score: {overall_f1_score}")

Overall Metrics:
Overall Precision: 0.8169449598021027
Overall Recall: 0.91991643454039
Overall F1-Score: 0.8653783164100884


In [ ]:
pprint.pp(class_metrics)

{'car': {'TP': 1184, 'FP': 216, 'FN': 71},
 'bus': {'TP': 16, 'FP': 17, 'FN': 16},
 'truck': {'TP': 121, 'FP': 63, 'FN': 28}}
